# In this notebook, we will explore the effect of the ratio of f to p on the model

In [ ]:
from simulations.drosselschwab import simulate_drosselschwab
import results
import data
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import csv
import os
from pathlib import Path
from datetime import datetime
from scripts.parallel_sims import worker
import numpy as np

In [ ]:
# Create experiments root under data/f_over_p and pick the next available experiment index
base_dir = Path("../data/f_over_p").resolve()
base_dir.mkdir(parents=True, exist_ok=True)
idx = 1
while (base_dir / f"experiment_{idx}").exists():
    idx += 1
outdir = base_dir / f"experiment_{idx}"
outdir.mkdir(parents=True, exist_ok=False)
print(f"Saving notebook run results to experiment directory: {outdir}")

In [ ]:
# Example parameter sweep: small demo grid. Replace with your actual sweep.
param_list = []
param_idx = 1
L = 256
steps = 5000

runs_per_param = 5

f_over_p_ratios = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]

p_values = [0.01, 0.05, 0.1]
f_values = [p_values[0]*f_over_p_ratio for f_over_p_ratio in f_over_p_ratios]

for p in p_values:
    for f in f_values:

        for run_idx in range(runs_per_param):
            param_list.append({'L': L, 'p': p, 'f': f, 'steps': steps, 'param_id': param_idx, 'run_id': run_idx})
        param_idx += 1

In [ ]:
param_list

In [ ]:
max_workers = int(os.environ.get('MAX_WORKERS', multiprocessing.cpu_count()))
print(f"Running {len(param_list)} simulations with up to {max_workers} workers...")

results = []
with ProcessPoolExecutor(max_workers=max_workers) as exe:
    # pass outdir as first argument to worker
    futures = {exe.submit(worker, outdir, params): params for params in param_list}
    for fut in as_completed(futures):
        params = futures[fut]
        try:
            res = fut.result()
            print(f"Done: p={res['p']}, f={res['f']}, fires={res['num_fires']}, mean={res['mean_size']:.2f}, max={res['max_size']}")
            results.append(res)
        except Exception as e:
            print(f"Error for params {params}: {e}")

# Write a summary CSV
summary_file = outdir / f"summary_{datetime.now().strftime('%Y%m%dT%H%M%SZ')}.csv"
keys = ['L', 'p', 'f', 'steps', 'param_id', 'run_id', 'num_fires', 'mean_size', 'max_size', 'remaining_trees', 'raw_file']
with open(summary_file, 'w', newline='') as fh:
    writer = csv.DictWriter(fh, keys)
    writer.writeheader()
    for r in results:
        writer.writerow({k: r.get(k, '') for k in keys})

print(f"Summary written to {summary_file}")

In [ ]:
# --- New analysis cells: load per-step CSVs from the latest experiment and compute statistics ---
import json
import csv
import re
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# auto-detect latest experiment directory under data/f_over_p
base_dir = Path("../data/f_over_p").resolve()
if not base_dir.exists():
    raise FileNotFoundError(f"Base data directory not found: {base_dir}")
exp_dirs = [d for d in base_dir.iterdir() if d.is_dir() and d.name.startswith('experiment_')]
if not exp_dirs:
    raise FileNotFoundError(f"No experiment directories found under {base_dir}")
def _exp_index(d):
    try:
        return int(d.name.split('_')[-1])
    except Exception:
        return -1
exp_dirs_sorted = sorted(exp_dirs, key=_exp_index)
EXP_DIR = exp_dirs_sorted[-1].resolve()
print('Experiment dir:', EXP_DIR)

# find perstep files created by the worker
perstep_files = sorted(EXP_DIR.glob('perstep_param*_*.csv'))
print(f'Found {len(perstep_files)} per-step files')

# filename pattern to extract param_id and run_id
pattern = re.compile(r'perstep_param(?P<param>\d+)_.*_id(?P<run>\d+)_')


def load_perstep_file(fp):
    records = []
    with open(fp, newline='') as fh:
        reader = csv.reader(fh)
        try:
            header = next(reader)
        except StopIteration:
            return records
        for row in reader:
            if not row:
                continue
            # expected columns: step, fire_size (JSON), cluster distr (JSON), mean tree density
            step = int(row[0])
            fires = json.loads(row[1]) if row[1] else []
            clusters = json.loads(row[2]) if row[2] else []
            density = float(row[3]) if row[3] != '' else None
            records.append({'step': step, 'fires': fires, 'clusters': clusters, 'density': density})
    return records

# organize runs by param_id
runs_by_param = {}  # param_id -> list of runs
for fp in perstep_files:
    m = pattern.search(fp.name)
    if not m:
        print('Skipping unknown file pattern:', fp.name)
        continue
    pid = int(m.group('param'))
    rid = int(m.group('run'))
    recs = load_perstep_file(fp)
    # aggregate series for convenience
    fires_all = []
    clusters_all = []
    density_series = []
    for r in recs:
        fires_all.extend(r['fires'])
        clusters_all.extend(r['clusters'])
        density_series.append(r['density'])
    runs_by_param.setdefault(pid, []).append({'run_id': rid, 'file': fp, 'records': recs, 'fires_all': fires_all, 'clusters_all': clusters_all, 'density_series': density_series})

print('Loaded runs for param_ids:', sorted(runs_by_param.keys()))


In [ ]:
# Plot: Fire-size distributions for individual runs with the same param_id
# Choose a param_id to inspect (if multiple present, this will loop over all)
for pid in sorted(runs_by_param.keys()):
    runs = runs_by_param[pid]
    if not runs:
        continue
    plt.figure(figsize=(7, 4))
    # compute global min/max across runs for consistent bins
    all_fs = np.concatenate([np.array(r['fires_all']) for r in runs if len(r['fires_all']) > 0]) if any(len(r['fires_all'])>0 for r in runs) else np.array([])
    if all_fs.size == 0:
        print(f'param_id {pid}: no fires recorded in any run')
        continue
    min_s = max(1, int(all_fs.min()))
    max_s = int(all_fs.max())
    bins = np.logspace(np.log10(min_s), np.log10(max_s), num=20)

    for r in runs:
        fs = np.array(r['fires_all'])
        if fs.size == 0:
            continue
        hist, edges = np.histogram(fs, bins=bins, density=True)
        centers = np.sqrt(edges[:-1] * edges[1:])
        mask = hist > 0
        plt.loglog(centers[mask], hist[mask], marker='o', linestyle='-', label=f"run {r['run_id']}")

    plt.title(f'Fire-size distributions for param_id {pid} (per-run)')
    plt.xlabel('Fire size')
    plt.ylabel('Probability density')
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# Plot: Fire-size distributions aggregated across runs for different param_ids
# compute global bins across all param_ids
all_fires_global = np.concatenate([np.concatenate([np.array(r['fires_all']) for r in runs_by_param[pid] if len(r['fires_all'])>0]) for pid in runs_by_param.keys() if any(len(r['fires_all'])>0 for r in runs_by_param[pid])]) if any(any(len(r['fires_all'])>0 for r in runs_by_param[pid]) for pid in runs_by_param.keys()) else np.array([])
if all_fires_global.size > 0:
    min_s = max(1, int(all_fires_global.min()))
    max_s = int(all_fires_global.max())
    bins = np.logspace(np.log10(min_s), np.log10(max_s), num=25)

    plt.figure(figsize=(8, 5))
    for pid in sorted(runs_by_param.keys()):
        runs = runs_by_param[pid]
        # aggregate fires across runs for this param_id
        agg = np.concatenate([np.array(r['fires_all']) for r in runs if len(r['fires_all']) > 0]) if any(len(r['fires_all'])>0 for r in runs) else np.array([])
        if agg.size == 0:
            continue
        hist, edges = np.histogram(agg, bins=bins, density=True)
        centers = np.sqrt(edges[:-1] * edges[1:])
        mask = hist > 0
        plt.loglog(centers[mask], hist[mask], marker='o', linestyle='-', label=f'param {pid}')

    plt.title('Fire-size distributions aggregated by param_id')
    plt.xlabel('Fire size')
    plt.ylabel('Probability density')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print('No fires in any run to plot aggregated distributions')


In [ ]:
# Plot: Cluster-size distributions (aggregated across runs and steps) per param_id
all_clusters_global = np.concatenate([np.concatenate([np.array(r['clusters_all']) for r in runs_by_param[pid] if len(r['clusters_all'])>0]) for pid in runs_by_param.keys() if any(len(r['clusters_all'])>0 for r in runs_by_param[pid])]) if any(any(len(r['clusters_all'])>0 for r in runs_by_param[pid]) for pid in runs_by_param.keys()) else np.array([])
if all_clusters_global.size > 0:
    min_c = max(1, int(all_clusters_global.min()))
    max_c = int(all_clusters_global.max())
    bins = np.logspace(np.log10(min_c), np.log10(max_c), num=25)

    plt.figure(figsize=(8, 5))
    for pid in sorted(runs_by_param.keys()):
        runs = runs_by_param[pid]
        agg = np.concatenate([np.array(r['clusters_all']) for r in runs if len(r['clusters_all']) > 0]) if any(len(r['clusters_all'])>0 for r in runs) else np.array([])
        if agg.size == 0:
            continue
        hist, edges = np.histogram(agg, bins=bins, density=True)
        centers = np.sqrt(edges[:-1] * edges[1:])
        mask = hist > 0
        plt.loglog(centers[mask], hist[mask], marker='o', linestyle='-', label=f'param {pid}')

    plt.title('Cluster-size distributions aggregated by param_id')
    plt.xlabel('Cluster size')
    plt.ylabel('Probability density')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print('No clusters recorded in any run')


In [ ]:
# Plot: Mean tree density time series per run and averaged per param_id
for pid in sorted(runs_by_param.keys()):
    runs = runs_by_param[pid]
    if not runs:
        continue
    plt.figure(figsize=(8, 4))
    # collect arrays of densities; pad shorter runs if needed
    densities = [np.array(r['density_series'], dtype=float) for r in runs if len(r['density_series'])>0]
    if not densities:
        print(f'param {pid}: no density series available')
        continue
    maxlen = max(arr.size for arr in densities)
    stacked = np.vstack([np.pad(arr, (0, maxlen - arr.size), constant_values=np.nan) for arr in densities])
    # plot individual runs
    for i, arr in enumerate(stacked):
        plt.plot(np.arange(arr.size), arr, alpha=0.3, label=f'run {runs[i]["run_id"]}')
    # mean and std across runs (ignore nan)
    mean_series = np.nanmean(stacked, axis=0)
    std_series = np.nanstd(stacked, axis=0)
    x = np.arange(mean_series.size)
    plt.plot(x, mean_series, color='k', linewidth=2, label='mean')
    plt.fill_between(x, mean_series - std_series, mean_series + std_series, color='k', alpha=0.2, label='std')
    plt.title(f'Mean tree density over time (param {pid})')
    plt.xlabel('Step')
    plt.ylabel('Mean tree density')
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# --- New comparison plots across parameter values ---
# Plot mean tree density over time compared across different parameter values (averaged over runs)
# and steady-state mean tree density per parameter (excluding first 100 steps)

# Build an optional map from param_id -> (p, f) from latest summary file for nicer labels
summary_map = {}
try:
    summaries = list(EXP_DIR.glob('summary_*.csv'))
    if summaries:
        latest = max(summaries, key=lambda p: p.stat().st_mtime)
        with open(latest, newline='') as fh:
            reader = csv.DictReader(fh)
            for row in reader:
                try:
                    pid = int(row.get('param_id', ''))
                except Exception:
                    continue
                pval = row.get('p', '')
                fval = row.get('f', '')
                summary_map[pid] = (pval, fval)
except Exception:
    summary_map = {}

# Prepare mean series per param_id (align by padding with NaN)
param_mean_series = {}
maxlen_global = 0
for pid in sorted(runs_by_param.keys()):
    runs = runs_by_param[pid]
    densities = [np.array(r['density_series'], dtype=float) for r in runs if len(r['density_series'])>0]
    if not densities:
        continue
    maxlen = max(arr.size for arr in densities)
    if maxlen > maxlen_global:
        maxlen_global = maxlen
    stacked = np.vstack([np.pad(arr, (0, maxlen - arr.size), constant_values=np.nan) for arr in densities])
    mean_series = np.nanmean(stacked, axis=0)
    param_mean_series[pid] = mean_series

if not param_mean_series:
    print('No parameter mean series available to plot comparison')
else:
    plt.figure(figsize=(10, 5))
    for pid, series in param_mean_series.items():
        # pad to global length
        s = np.pad(series, (0, maxlen_global - series.size), constant_values=np.nan)
        label = f'param {pid}'
        if pid in summary_map:
            pval, fval = summary_map[pid]
            label = f'param {pid} (p={pval}, f={fval})'
        plt.plot(np.arange(maxlen_global), s, label=label)
    plt.xlabel('Step')
    plt.ylabel('Mean tree density')
    plt.title('Mean tree density over time (averaged over runs) - comparison across parameter values')
    plt.legend(loc='best', fontsize='small')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Steady-state comparison (exclude first 100 steps)
STEADY_IGNORE = 100
steady_means = []
steady_stds = []
steady_labels = []
f_vals = []
for pid in sorted(runs_by_param.keys()):
    runs = runs_by_param[pid]
    per_run_steady = []
    for r in runs:
        arr = np.array(r['density_series'], dtype=float)
        if arr.size <= STEADY_IGNORE:
            continue
        val = np.nanmean(arr[STEADY_IGNORE:])
        if not np.isnan(val):
            per_run_steady.append(val)
    if not per_run_steady:
        continue
    steady_means.append(np.mean(per_run_steady))
    steady_stds.append(np.std(per_run_steady))
    lbl = f'param {pid}'
    if pid in summary_map:
        pval, fval = summary_map[pid]
        lbl = f'param {pid} (p={pval}, f={fval})'
    steady_labels.append(lbl)
    f_vals.append(float(fval) if fval != '' else np.nan)

if not steady_means:
    print('No steady-state statistics available (not enough steps after ignoring first 100)')
else:
    x = np.arange(len(steady_means))
    plt.figure(figsize=(8, 4))
    plt.plot(f_vals, steady_means, alpha=0.8)
    # plt.plot(x, steady_means, yerr=steady_stds, capsize=6, alpha=0.8)
    # plt.xticks(x, steady_labels, rotation=45, ha='right')
    plt.xscale('log')
    plt.ylabel('Steady-state mean tree density (mean over runs)')
    plt.title(f'Steady-state mean tree density per parameter (excluding first {STEADY_IGNORE} steps)')
    plt.tight_layout()
    plt.show()


In [ ]:
# End of analysis cells
print('Analysis complete')
